# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [2]:
import os

import numpy as np
import datetime
import matplotlib.pyplot as plt

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers


import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, RandomFlip, RandomRotation
from tensorflow.keras.utils import Sequence

 

/home/kevin/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kevin/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/kevin/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below

GPU limitation

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-08-09 11:12:30.721105: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-09 11:12:30.757049: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-09 11:12:30.757300: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-09 11:12:30.758301: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Building the Models

## Optimized Model 1

In [202]:
#cut a video into different frames
def cut_video(filename):
    list_of_frames = []
    cap = cv2.VideoCapture(filename)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count  = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        list_of_frames.append(frame)
   
    cap.release()
    return np.array(list_of_frames)


# loop through each frame in the array of images, take two frames at a time, 
# # loop through each pixel in the 2 frames, find the differences in values 
# in grey scale for the corresponding pixels, append the diff to the rgb array 
# representing each pixel =>>>>>>>> RGBX. Notei if first frame append 0 to rgb array
def load_images_from_folders(data_path,total): 
    images = np.zeros((total,224,224,4),dtype=float)
    counter = 0

    #loop through each vid in the folder 
    for index in range(len(data_path)): #looping through the entire folder array
        
        #variable to track whether it is the first image/frame in the video
        first_image = True

        #cut the video into different frames and save them all into an array
        array_images = cut_video(data_path[index]) # array_images[0][0][0] = single pixel

        #array to store modified pixel of each image, later on append it to images[]
        
  
        for i in range(len(array_images)-1):
            if first_image == True:
                #process for 0th and 1st frame
                images[counter] = first_frame_pixel_modification(array_images[i])
                first_image = False
                counter+=1
            
            images[counter] = two_frame_pixel_modification(array_images[i],array_images[i+1])
            counter+=1
        
    #         print("finished frame " + str(i)+ " of video " + str(index))
        
    #     print(str(counter) +" matches " + str(len(array_images)))
    #     print("finished video " + str(index))
        
    #     #after finish loop through a frame, append the frame into the images[] array
    # print("final count: " + str(counter) + "; expected count = " + str(total))
    print(counter)
    # print(images.shape)
    return images

def two_frame_pixel_modification(frame1, frame2): 
    row_frame = np.zeros((224,224,4),dtype=float)
    np.array(row_frame) 
    for r in range(len(frame1)):
        column_frame = np.zeros((224,4),dtype=float)
        for c in range(len(frame1[0])):
            #calcualte each pixel's difference 
            grey1 =  find_greyscale(frame1[r][c][2],frame1[r][c][1],frame1[r][c][0])
            grey2 =  find_greyscale(frame2[r][c][2],frame2[r][c][1],frame2[r][c][0])
            diff_grey = abs(grey1-grey2)
            new_rgbx = [frame2[r][c][2],frame2[r][c][1],frame2[r][c][0],diff_grey] #add greyscale diff to the end of the pixel array, and append it to the individual frame array
            new_rgbx = np.array(new_rgbx)
            column_frame[c] = np.array(new_rgbx)
        
        row_frame[r] = column_frame
 
        
    return row_frame 

def first_frame_pixel_modification(frame0):
    row_frame = np.zeros((224,224,4),dtype=float)
    np.array(row_frame)
    for r in range(len(frame0)):

        column_frame = np.zeros((224,4),dtype=float)
        for c in range(len(frame0[0])): 

            new_rgbx = [frame0[r][c][2],frame0[r][c][1],frame0[r][c][0],0]
            column_frame[c] = np.array(new_rgbx)

        row_frame[r] = column_frame

    return row_frame
    

def find_greyscale(r,g,b):
    grey = 0.299*r + 0.587*g + 0.114*b
    return grey

def cal_totalframes(array_of_vids):
    total_frames = 0
    for i in range(len(array_of_vids)):
        list_of_cut_frames = cut_video(array_of_vids[i])
        total_frames+= len(list_of_cut_frames)
    return total_frames






moving_path = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

still_path = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]

moving_data = load_images_from_folders(moving_path,cal_totalframes(moving_path))
still_data = load_images_from_folders(still_path,cal_totalframes(still_path))




513
567


### Prepare data

In [312]:
#concatenate still and movind data to create inputs and outputs
inputs = np.concatenate((still_data, moving_data))

outputs = np.concatenate((np.zeros(len(still_data)),np.ones(len(moving_data))))


#shuffle the data to prepare for training 
shuffled_indices = np.random.permutation(len(inputs))
inputs, outputs = inputs[shuffled_indices],outputs[shuffled_indices]

#inputs = np.concatenate((inputs,inputs))
#outputs = np.concatenate((outputs,outputs))

# inputs = inputs.astype(np.float32)
# outputs = outputs.astype(np.float32)
inputs = inputs / 255.0
outputs = outputs.astype(np.float32)
print(inputs[0][0][0])

#create a tensorflow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((inputs,outputs))
    dataset = dataset.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

print(inputs.shape)
print(outputs.shape)
print(len(dataset))

#determine train and test 
train_data = dataset.take(int(1.0*len(dataset)))

[0.65882353 0.65098039 0.69019608 0.00481569]
(1080, 224, 224, 4)
(1080,)
34


In [313]:
print(inputs.shape)
print(outputs.shape)


(1080, 224, 224, 4)
(1080,)


### Define Model

In [314]:
# input_shape = (2160,224, 224, 4)
# input_shape = (1, 224, 224, 4)
input_shape = (None, 224, 224, 4)

#data augmentation 
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomContrast(factor=0.1)

])


model = tf.keras.Sequential([

    # Add Convolutional and Pooling layers
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten the output of the Convolutional layers before passing to Dense layers
    Flatten(),
    
    # Add Dense layers for classification
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid'),  # Using sigmoid activation for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Print the model summary to verify the input shape
model.build(input_shape)
model.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_26 (Sequential)  (None, 224, 224, 4)       0         
                                                                 
 conv2d_39 (Conv2D)          (None, 222, 222, 32)      1184      
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_40 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 52, 52, 128)     

### Train Model

In [315]:
checkpoint_filepath = 'tmp/checkpoints'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


#model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[model_checkpoint_callback])
model.fit(train_data, epochs=10, validation_data=tdataset, callbacks=[model_checkpoint_callback])
#model.fit(train_data, epochs=5)

model.load_weights(checkpoint_filepath)


Epoch 1/10
34/34 [==============================] - 5s 139ms/step - loss: 0.5602 - accuracy: 0.7250 - val_loss: 0.3822 - val_accuracy: 0.8187
Epoch 2/10
34/34 [==============================] - 3s 100ms/step - loss: 0.2531 - accuracy: 0.8981 - val_loss: 0.3961 - val_accuracy: 0.8158
Epoch 3/10
34/34 [==============================] - 3s 102ms/step - loss: 0.2795 - accuracy: 0.8843 - val_loss: 0.4521 - val_accuracy: 0.7704
Epoch 4/10
34/34 [==============================] - 3s 100ms/step - loss: 0.1870 - accuracy: 0.9204 - val_loss: 0.5208 - val_accuracy: 0.7655
Epoch 5/10
34/34 [==============================] - 3s 102ms/step - loss: 0.1546 - accuracy: 0.9259 - val_loss: 0.5093 - val_accuracy: 0.7232
Epoch 6/10
34/34 [==============================] - 3s 99ms/step - loss: 0.1918 - accuracy: 0.9046 - val_loss: 0.4753 - val_accuracy: 0.7586
Epoch 7/10
34/34 [==============================] - 3s 100ms/step - loss: 0.1580 - accuracy: 0.9389 - val_loss: 0.6612 - val_accuracy: 0.7438
Epoch 8

In [286]:
pos = 0
neg = 0
for x, y in train_data.unbatch():
    #print(x[0][0],y)
    if y > 0.5:
        pos += 1
    else:
        neg += 1

print(pos, neg)

822 906


### Evaluate on test dataset

In [207]:
###TESTING 
test_moving = [ "./../datasets/vids/testdata/moving/resized-test-fall1.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall2.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall3.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall4.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall5.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall6.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall7.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall8.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall9.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall10.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall11.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall12.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall13.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall14.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall15.mp4"
]

test_still = [
                "./../datasets/vids/testdata/still/resized-test-default1.mp4",
                "./../datasets/vids/testdata/still/resized-test-default2.mp4",
                "./../datasets/vids/testdata/still/resized-test-default3.mp4",
                "./../datasets/vids/testdata/still/resized-test-default4.mp4",
                "./../datasets/vids/testdata/still/resized-test-default5.mp4",
                "./../datasets/vids/testdata/still/resized-test-default6.mp4",
                "./../datasets/vids/testdata/still/resized-test-default7.mp4",
                "./../datasets/vids/testdata/still/resized-test-default8.mp4",
                "./../datasets/vids/testdata/still/resized-test-default9.mp4",
                "./../datasets/vids/testdata/still/resized-test-default10.mp4",
                "./../datasets/vids/testdata/still/resized-test-default11.mp4",
                "./../datasets/vids/testdata/still/resized-test-default12.mp4",
                "./../datasets/vids/testdata/still/resized-test-default13.mp4",
                "./../datasets/vids/testdata/still/resized-test-default14.mp4",
                "./../datasets/vids/testdata/still/resized-test-default15.mp4"]

still_tdata = load_images_from_folders(test_still,cal_totalframes(test_still))
moving_tdata = load_images_from_folders(test_moving,cal_totalframes(test_moving))




501
514


In [316]:
#concatenate still and movind data to create inputs and outputs
tinputs = np.concatenate((still_tdata, moving_tdata))
toutputs = np.concatenate((np.zeros(len(still_tdata)),np.ones(len(moving_tdata))))

# tinputs = tinputs.astype(np.float32)
# toutputs = toutputs.astype(np.float32)

tinputs = tinputs / 255.0
toutputs = toutputs.astype(np.float32)
#shuffle the data to prepare for training 
tshuffled_indices = np.random.permutation(len(tinputs))
tinputs, toutputs = tinputs[tshuffled_indices],toutputs[tshuffled_indices]

print(tinputs)
print(toutputs)

#create a tensorflow dataset
with tf.device('/cpu:0'):
    tdataset = tf.data.Dataset.from_tensor_slices((tinputs,toutputs))
    tdataset = tdataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)


# model.summary()

[[[[0.49803922 0.51372549 0.52941176 0.        ]
   [0.49803922 0.51372549 0.52941176 0.        ]
   [0.49411765 0.50980392 0.5254902  0.        ]
   ...
   [0.16078431 0.10980392 0.09019608 0.        ]
   [0.14117647 0.09803922 0.07843137 0.        ]
   [0.14117647 0.09803922 0.07843137 0.        ]]

  [[0.51764706 0.53333333 0.54901961 0.        ]
   [0.51764706 0.53333333 0.54901961 0.        ]
   [0.51372549 0.52941176 0.54509804 0.        ]
   ...
   [0.15686275 0.10588235 0.08627451 0.        ]
   [0.13333333 0.09019608 0.07058824 0.        ]
   [0.13333333 0.09019608 0.07058824 0.        ]]

  [[0.54117647 0.55686275 0.57254902 0.        ]
   [0.54117647 0.55686275 0.57254902 0.        ]
   [0.53333333 0.54901961 0.56470588 0.        ]
   ...
   [0.16470588 0.10196078 0.07843137 0.        ]
   [0.1372549  0.08627451 0.05882353 0.        ]
   [0.1372549  0.08627451 0.05882353 0.        ]]

  ...

  [[0.4627451  0.48235294 0.50588235 0.        ]
   [0.4627451  0.48235294 0.5058823

In [317]:
# model.load_weights(checkpoint_filepath)
loss, accuracy = model.evaluate(tdataset)
#loss, accuracy = model.evaluate(train_data.take(100))


64/64 [==============================] - 2s 15ms/step - loss: 0.3822 - accuracy: 0.8187


### save and quantize model, turn into tflite

In [318]:
#save model1 
model.save("optimized_model1")

INFO:tensorflow:Assets written to: optimized_model1/assets


INFO:tensorflow:Assets written to: optimized_model1/assets


In [319]:
def representative_dataset():
    for data, label in tf.data.Dataset.from_tensor_slices((inputs, outputs)).batch(1).take(100):
        yield [tf.dtypes.cast(data, tf.float32)]

with tf.device('/cpu:0'):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    tflite_quant_m1 = converter.convert()

    # Save the quantized model 1
    with open('optimized_model1.tflite', 'wb') as f: f.write(tflite_quant_m1)

INFO:tensorflow:Assets written to: /tmp/tmpjkjdc6qj/assets


INFO:tensorflow:Assets written to: /tmp/tmpjkjdc6qj/assets
/home/kevin/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-08-08 16:33:33.801114: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-08-08 16:33:33.801138: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-08-08 16:33:33.828848: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpjkjdc6qj
2023-08-08 16:33:33.850096: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-08-08 16:33:33.850160: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpjkjdc6qj
2023-08-08 16:33:33.955278: I tensorflow/cc/saved_model/loader.cc:2

### Code to separate inputs of each video

In [320]:


moving_path = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

still_path = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]


def cal_total_idvFrames(vid):
    total_frame = len(cut_video(vid))
    return total_frame

def separate_aggregated_images(aggregated_images, num_images_per_video):
    separated_arrays = []
    start_idx = 0

    for num_images in num_images_per_video:
        end_idx = start_idx + num_images
        video_images = aggregated_images[start_idx:end_idx]
        separated_arrays.append(video_images)
        start_idx = end_idx

    return np.array(separated_arrays)

def separate_inputs(inputs,path):
    list_total_of_each_vid = []

    for i in range(len(path)):
        list_total_of_each_vid.append(cal_total_idvFrames(path[i]))
    
    #once obtained total of each vid, separate each
    updated_inputs = separate_aggregated_images(inputs,list_total_of_each_vid)
    return np.array(updated_inputs)


separated_moving_inputs = separate_inputs(moving_data,moving_path)
separated_still_inputs = separate_inputs(still_data,still_path)

print(separated_moving_inputs)
print(separated_still_inputs.shape)


[array([[[[156.   , 155.   , 160.   ,   0.   ],
          [156.   , 155.   , 160.   ,   0.   ],
          [156.   , 155.   , 160.   ,   0.   ],
          ...,
          [111.   , 111.   , 111.   ,   0.   ],
          [110.   , 110.   , 110.   ,   0.   ],
          [110.   , 110.   , 110.   ,   0.   ]],

         [[156.   , 155.   , 160.   ,   0.   ],
          [156.   , 155.   , 160.   ,   0.   ],
          [156.   , 155.   , 160.   ,   0.   ],
          ...,
          [111.   , 111.   , 111.   ,   0.   ],
          [110.   , 110.   , 110.   ,   0.   ],
          [110.   , 110.   , 110.   ,   0.   ]],

         [[156.   , 155.   , 160.   ,   0.   ],
          [156.   , 155.   , 160.   ,   0.   ],
          [156.   , 155.   , 160.   ,   0.   ],
          ...,
          [111.   , 111.   , 111.   ,   0.   ],
          [110.   , 110.   , 110.   ,   0.   ],
          [110.   , 110.   , 110.   ,   0.   ]],

         ...,

         [[ 87.   ,  71.   ,  54.   ,   0.   ],
          [ 87.   ,  7

/tmp/ipykernel_607854/1747160067.py:52: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(separated_arrays)


### Use model 1 to perform prediction and generate dataset for model 2

In [327]:
with tf.device('/cpu:0'):

    def batchPredict(list_of_images_of_a_singlevid):
        probs = []

        # define intepreter for quantized modell
        intepreter = tf.lite.Interpreter(model_path="optimized_model1.tflite")
        intepreter.allocate_tensors()
        #get outputs and inputs details
        output = intepreter.get_output_details()
        input = intepreter.get_input_details()
        output_index = output[0]['index']
        input_index = input[0]['index']
        
     
        # output_index = output[0]['index']
        # input_index = input[0]['index']
        
        #set input ->invoke -> access outputs
        list_of_images_of_a_singlevid = np.array(list_of_images_of_a_singlevid, dtype=np.uint8)

        #run through model and predict the output
        for instance in list_of_images_of_a_singlevid:
            intepreter.set_tensor(input_index, np.expand_dims(instance,axis=0))
            intepreter.invoke()
            output_data = intepreter.get_tensor(output_index)
            output_data = output_data[0][0]
            # print(output_data)
            probs.append(output_data)
        
        return probs
            
        

    def batchPredictMultVids(list_of_images_of_all_vids):
        record = []
        for i in range(len(list_of_images_of_all_vids)):
            vid_probs = batchPredict(list_of_images_of_all_vids[i])
            #append sliding frames of 8 into the record
            for f in range(len(vid_probs)-7):
                record.append(vid_probs[f:f+8])
            print(len(record))
        return np.array(record)
            
    

with tf.device('/cpu:0'):
    falling_frames = []
    default_frames = []

    falling_frames = batchPredictMultVids(separated_moving_inputs)
    default_frames = batchPredictMultVids(separated_still_inputs)

   

[[18, 18, 13, 18, 14, 17, 20, 48], [18, 13, 18, 14, 17, 20, 48, 91], [13, 18, 14, 17, 20, 48, 91, 117], [18, 14, 17, 20, 48, 91, 117, 165], [14, 17, 20, 48, 91, 117, 165, 241], [17, 20, 48, 91, 117, 165, 241, 244], [20, 48, 91, 117, 165, 241, 244, 225], [48, 91, 117, 165, 241, 244, 225, 246], [91, 117, 165, 241, 244, 225, 246, 160], [117, 165, 241, 244, 225, 246, 160, 255], [165, 241, 244, 225, 246, 160, 255, 255], [241, 244, 225, 246, 160, 255, 255, 222], [244, 225, 246, 160, 255, 255, 222, 222], [225, 246, 160, 255, 255, 222, 222, 235], [246, 160, 255, 255, 222, 222, 235, 255], [160, 255, 255, 222, 222, 235, 255, 255], [255, 255, 222, 222, 235, 255, 255, 63], [255, 222, 222, 235, 255, 255, 63, 255], [222, 222, 235, 255, 255, 63, 255, 106], [222, 235, 255, 255, 63, 255, 106, 235], [235, 255, 255, 63, 255, 106, 235, 236], [255, 255, 63, 255, 106, 235, 236, 106], [255, 63, 255, 106, 235, 236, 106, 106], [63, 255, 106, 235, 236, 106, 106, 180], [255, 106, 235, 236, 106, 106, 180, 101], [

### Save data

In [330]:


file2 = open("model1_output_default.txt", "w")
file2.write(str(default_frames))
file2.close()

file1 = open("model1_output_fall.txt", "w")
file1.write(str(falling_frames))
file1.close()

In [9]:
default_data = np.loadtxt('model1_output_default.txt')
falling_data = np.loadtxt('model1_output_fall.txt')
print(falling_data.shape)
print(default_data.shape)

(408, 8)
(462, 8)


## Model 2

### Process data

In [11]:
# loading data
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data)))) #ones are falling , zeros are default
dataset_size = len(inputs)
new_indices = np.random.permutation(dataset_size) # shuffle indices to shuffle X and y at the same time
inputs, outputs = inputs[new_indices], outputs[new_indices]

train_size = int(0.8*dataset_size)
test_size = dataset_size - train_size


# train_data = tf.data.Dataset.from_tensor_slices((inputs[:train_size], outputs[:train_size])).batch(128)
X_train, y_train = inputs[:train_size], outputs[:train_size] #x = images, y = label 
X_test, y_test = inputs[train_size:], outputs[train_size:]

In [10]:
# loading data, output is used to validate input
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data)))) #ones are falling , zeros are default


dataset_size = len(inputs) #length of the entire dataset, including both outputs and inputs
new_indices = np.random.permutation(dataset_size) # using dataset size, shufflen the indices to shuffle X and y at the same time
print(new_indices)

#new_indices is an array that contains all indices of the dataset in a shuffled order, when passed into inputs/outputs, other np arrays, inputs and outputs will be rearranged to match the index of the new_indices array
inputs, outputs = inputs[new_indices], outputs[new_indices] 


train_size = int(0.8*dataset_size)
test_size = dataset_size - train_size


# train_data = tf.data.Dataset.from_tensor_slices((inputs[:train_size], outputs[:train_size])).batch(128)
X_train, y_train = inputs[:train_size], outputs[:train_size] #x = images, y = label 
X_test, y_test = inputs[train_size:], outputs[train_size:]


# Define the model architecture
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(100, activation="relu"),
    Dense(100, activation="relu"),
    Dense(10, activation="sigmoid")
])

# Compile the model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])

# Train the model
history = model.fit(X_train, y_train,
                    batch_size=32, epochs=50,
                    validation_data=(X_test, y_test), verbose=0)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Plot the validation accuracy over epochs
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()




[485 158 510 865 501  43  66  34 553 418 403  14 798  16 104   0 676 597
 759 363 386 648  22  95 374 655 582   5 339 179 224 815 758 605 558 157
 124 113 167 704 247 347  56 450 357 784 204 441 219 775 772 773 192 362
 812 755 292 576   3 351  60 603 663 464 566 794  71 494 546 342 740 500
 841 250 406 184 774 720 451 118 254 722 670 854 612 134 349  53 146 716
 216  25  88 325 669 489 642 866 647 285 273 822 797 227 151 181 623 168
 319 833 744 255 738 433 196 562 317  97 803 329 733  79 536 832 436 581
 752 195 680 223 466 242 162  55  75  30 678 256 625 393 791 318 813  80
 786 125 692 248 596 315 664 782 346 551 632 207 144 696  18 131 145 544
  36  94 531 395 416 370 132 379 306 442 585 610 445 200 532 202 508 668
 111 525 509 147 575 507 284 837 206 114 197 570 778 149 280  77 372 291
 415 323 316 569 313  27 847 844 843 613 498 296 695 518 367 355 272  28
 183 413 435 301 235  54 662   7 760 561 182 809  96 352 262 397 788 711
 586 234 750 472 439 399 617 457 264 657 864 724 83

ValueError: in user code:

    File "/home/kevin/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/kevin/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/kevin/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/kevin/.local/lib/python3.8/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/home/kevin/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/kevin/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'I

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.fit(x=X_train, y=y_train, epochs=6,callbacks=[tensorboard_callback])

### Evaluate & Export

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
# model.summary()
# model.save("model2")

In [ ]:
### CONVERT TO TFLITE
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model2_tflite = converter.convert()
with open('model2.tflite', 'wb') as f: f.write(model2_tflite)

In [ ]:
### QUANTIZE
def representative_dataset():
  for d in inputs:
    # d = np.expand_dims(d, axis=0)
    yield [tf.dtypes.cast(d, tf.float32)]

# print(dataset.cardinality().numpy())
# print(tf.shape(dataset))
# model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model2 = converter.convert()
with open('model2_quant', 'wb') as f: f.write(tflite_quant_model2)

# Predict

## Reformat Input

In [ ]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

In [ ]:
# Resize vid
video_path = "compilation_cut.mp4"
output_path = "compilation_cut_resized.mp4"
target_width = 224
target_height = 224

#resize_video(video_path, output_path, target_width, target_height)

# Load resized vid
cap = cv2.VideoCapture(output_path)
# frame_rate = 30
# cap.set(cv2.CAP_PROP_FPS, frame_rate) DOESN'T WORK
print(cap.get(cv2.CAP_PROP_FPS))

## Pass into model 1

In [ ]:
# Initialize list to store the frame classifications
frame_classifications = []
moving_prob = []
# Loop through the frames of the video (need to change to 30 fps)
while True:
    ret, frame = cap.read() 
    #just need to figure out if this is 30 fps

    if not ret: # Break the loop if the video has ended
        break

    ''' Formulate Input Data (frame_rgb) '''
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make frame input data and ensure its type matches the model
    frame_rgb = np.expand_dims(frame_rgb, axis=0)

    ''' Classify the Frame '''
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()

    # get_output_details() and get_input_details() return list of dictionaries of tensor details
    # keys: name, index, shape, shape_signature, dtype, quantization, ...
    # len(input) = len(output) = 1, so access the first element
    output = interpreter.get_output_details()
    input = interpreter.get_input_details()
    output_index = output[0]['index']
    input_index = input[0]['index']

    # set input -> invoke -> access output
    interpreter.set_tensor(input_index, frame_rgb)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_index)
    # If the output_data shape is (batch_size, num_classes), select the first frame
    output_data = output_data[0]

    # Convert each entry into probability
    output_probs = tf.nn.softmax(output_data.astype(float))

    # Find the index of the highest probability
    predicted_index = np.argmax(output_data)

    # Assuming you have a list of class labels corresponding to the model's output classes
    class_labels = ["Moving", "Still"]

    # Get the predicted class label
    predicted_class = class_labels[predicted_index]

    # Print the predicted class label
    # print("Predicted Class:", predicted_class)
    frame_classifications.append((predicted_class, max(output_probs.numpy())))
    
    prob = np.around(max(output_probs.numpy()), decimals = 2)
    if predicted_class == "Still":
        
        moving_prob.append(np.subtract(1, prob))
    else:
        moving_prob.append(prob)

## Prepare data for model 2

In [ ]:
moving_probs_trimmed = moving_prob[:-(len(moving_prob)%16)]
model2_in = np.array(moving_probs_trimmed).reshape((len(moving_prob)//16, 16))
print(model2_in)

## Pass into Model 2 and get final prediction

In [ ]:
model2 = tf.keras.models.load_model("model2")

In [ ]:
vid_preds = model2.predict(model2_in)

In [ ]:
threshold = 0.88
bools = vid_preds.reshape((1, len(vid_preds))) > threshold
print(bools)
print(vid_preds.reshape((1, len(vid_preds))))